In [91]:
using HumanoidLCMSim
using RigidBodySim
using RigidBodyDynamics
using ValkyrieRobot
using OrdinaryDiffEq
using DiffEqCallbacks
using RigidBodyTreeInspector
using ValkyrieRobot.BipedControlUtil
using RigidBodyDynamics.Contact

In [92]:
# controllerproc = spawn(`/home/twan/code/drake/bazel-bin/examples/humanoid_controller/valkyrie_balancing_demo`)

In [93]:
val = Valkyrie()
mechanism = val.mechanism
remove_fixed_tree_joints!(mechanism);

In [94]:
# add environment
rootframe = root_frame(mechanism)
ground = HalfSpace3D(Point3D(rootframe, 0., 0., 0.), FreeVector3D(rootframe, 0., 0., 1.))
add_environment_primitive!(mechanism, ground);

In [95]:
# Create visualizer
any_open_visualizer_windows() || (new_visualizer_window(); sleep(1));
vis = Visualizer()[:valkyrie]
setgeometry!(vis, mechanism, RigidBodyTreeInspector.parse_urdf(ValkyrieRobot.urdfpath(), mechanism; package_path = [ValkyrieRobot.packagepath()]))

In [96]:
feet = Dict(:left => findbody(mechanism, "leftFoot"), :right => findbody(mechanism, "rightFoot"))
hands = Dict(:left => findbody(mechanism, "leftPalm"), :right => findbody(mechanism, "rightPalm"))
robot_info = HumanoidRobotInfo(mechanism, feet, hands, parse_actuators(mechanism, ValkyrieRobot.urdfpath()));

In [97]:
state = MechanismState(mechanism)

MechanismState{Float64, Float64, Float64, …}(…)

In [98]:
function initialize(state::MechanismState, val::Valkyrie, vis::Visualizer)
    zero!(state)
    for side in instances(Side)
        set_configuration!(state, findjoint(mechanism, "$(side)KneePitch"), [1.205])
        set_configuration!(state, findjoint(mechanism, "$(side)HipPitch"), [-0.49])
        set_configuration!(state, findjoint(mechanism, "$(side)AnklePitch"), [-0.71])
        set_configuration!(state, findjoint(mechanism, "$(side)ShoulderPitch"), [0.300196631343025])
        set_configuration!(state, findjoint(mechanism, "$(side)ShoulderRoll"), [flipsign_if_right(-1.25, side)])
        set_configuration!(state, findjoint(mechanism, "$(side)ElbowPitch"), [flipsign_if_right(-0.785398163397448, side)])
        set_configuration!(state, findjoint(mechanism, "$(side)ForearmYaw"), [1.571])
    end
    set_configuration!(state, val.basejoint, [1; 0; 0; 0; 0; 0; 1.025])
    settransform!(vis, state)
    nothing
end

initialize (generic function with 1 method)

In [99]:
lcmcontroller = LCMController(robot_info)
controller = PeriodicController(1 / 300, lcmcontroller)
initialize(state, val, vis)
final_time = 10.
problem = ODEProblem(state, (0., final_time), controller)
@time sol = solve(problem, Tsit5(), abs_tol = 1e-10, dt = 0.05, callback = CallbackSet(vis, state, max_fps = 30.));

 20.526079 seconds (4.03 M allocations: 266.097 MiB, 1.73% gc time)


In [100]:
# Profile.clear()
# problem = ODEProblem(state, (0., 2.), controller)
# @time(@profile sol = solve(problem, Tsit5(), abs_tol = 1e-10, dt = 0.05, callback = CallbackSet(vis, state, max_fps = 30.)));
# using ProfileView
# ProfileView.view()

In [101]:
animate(vis, state, sol)

In [102]:
mean(diff(sol.t)) * 1e3

0.8173273395995097